In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '../terrace')

%load_ext autoreload
%autoreload 2

In [76]:
from tqdm import tqdm
from copy import deepcopy
import torch
from common.cfg_utils import get_config
from datasets.make_dataset import *
from models.gnn_bind import *
from terrace.batch import *
from terrace.type_data import *
from terrace.comp_node import *


cfg = get_config("../configs", "default")
loader = make_dataloader(cfg, "val")
in_node = Input(loader.get_type_data())

In [21]:
batch = next(iter(loader))

In [77]:
rot, _ = torch.linalg.qr(torch.randn((3,3)))
trans = torch.randn(3,)
# torch.linalg.det(rand_rot)
# torch.einsum("ij,jk->ik")
coord = batch.lig.ndata.coord
trans_coord = torch.einsum('ij,bj->bi',rot,coord) + trans

In [78]:
batch_t = deepcopy(batch)
batch_t.lig.ndata.coord = trans_coord

In [95]:
lig_coord = batch.lig.ndata.coord
rec_coord = batch.rec.ndata.coord
dists = torch.zeros((lig_coord.shape[0], rec_coord.shape[0]))
for i, lc in enumerate(lig_coord):
    for j, rc in enumerate(rec_coord):
        dists[i,j] = torch.linalg.norm(lc - rc)
dists

tensor([[24.9056, 26.1221, 32.7037,  ..., 59.9716, 57.3830, 72.9151],
        [25.5268, 26.5024, 33.3077,  ..., 58.4674, 55.8833, 71.4241],
        [26.6782, 27.3604, 34.4925,  ..., 57.7209, 55.0941, 70.8719],
        ...,
        [34.9228, 35.8056, 41.8239,  ..., 46.7359, 44.6136, 58.4612],
        [35.0984, 35.6710, 42.1213,  ..., 46.3110, 44.0902, 58.4083],
        [30.1799, 31.6070, 37.5665,  ..., 54.0163, 51.6679, 66.3070]])

In [97]:
mask = torch.zeros((lig_coord.shape[0], rec_coord.shape[0]))
tot_rec = 0
tot_lig = 0
rec_graph = batch.rec.dgl_batch
lig_graph = batch.lig.dgl_batch
for r, l in zip(rec_graph.batch_num_nodes(), lig_graph.batch_num_nodes()):
    mask[tot_lig:tot_lig+l,tot_rec:tot_rec+r] = 1.0
    tot_rec += r
    tot_lig += l
mask

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.]])

In [108]:
(1/(torch.cdist(lig_coord, rec_coord))**2)*mask

tensor([[0.0016, 0.0015, 0.0009,  ..., 0.0000, 0.0000, 0.0000],
        [0.0015, 0.0014, 0.0009,  ..., 0.0000, 0.0000, 0.0000],
        [0.0014, 0.0013, 0.0008,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0005, 0.0005, 0.0003],
        [0.0000, 0.0000, 0.0000,  ..., 0.0005, 0.0005, 0.0003],
        [0.0000, 0.0000, 0.0000,  ..., 0.0003, 0.0004, 0.0002]])

In [83]:
# from datasets.graphs.plot_graph import *
# plot_graph(batch_t.lig[0])